In [ ]:
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

# install findspark using pip
!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 281.3 MB 39 kB/s 
     |████████████████████████████████| 198 kB 52.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=de656b63e8371a14717e0dfcaacf842ce29b97f00765543a796a84552ca53b81
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


Retail Data Analytics

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import findspark
findspark.init()

In [ ]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
df = spark.read.option("delimiter", ";").option("inferSchema", "true").csv("retail-data.csv")

In [ ]:
customerAmount = df.select(col("_c3").alias("CustomerID"),col("_c2").alias("Amount"))

In [ ]:
results = customerAmount.groupBy("CustomerID").avg("Amount").withColumnRenamed("avg(Amount)", "AvgAmount").orderBy("CustomerID")

In [ ]:
results.show()

Social Media Content Wrangling

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, explode, arrays_zip
from pyspark.sql.types import ArrayType, StructType
import findspark
findspark.init()

In [ ]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
akunTwitter = spark.read.option("header", "true").option("inferSchema", "true").csv("AkunTwitter_POS.csv")

In [ ]:
hashtagTwitter = spark.read.option("header", "true").option("inferSchema", "true").csv("HashtagTwitter_POS.csv")

In [ ]:
instagram = spark.read.option("header", "true").option("inferSchema", "true").json("Instagram_POS.json")

In [ ]:
akun = akunTwitter.select("username", col("tweet").alias("content")).withColumn("source", lit("Twitter"))

In [ ]:
hashtag = hashtagTwitter.select("username", col("tweet").alias("content")).withColumn("source", lit("Twitter"))

In [ ]:
ig = instagram.select("caption","comments.author","comments.comment")

In [ ]:
igAuthorComment = ig.withColumn("comments", arrays_zip("author", "comment")) \
.withColumn("comments", explode("comments")) \
.select(col("comments.author").alias("username"), col("comments.comment").alias("content")) \
.withColumn("source", lit("Instagram"))

In [ ]:
joinAll = akun.unionAll(hashtag).unionAll(igAuthorComment)

In [ ]:
joinAll.coalesce(1).write.option("header", "true").csv("joinAll.csv")

In [ ]:
spark.stop()

SparkSQL Data Analytics

In [ ]:
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

findspark.init()

In [ ]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
schema = StructType([ \
                     StructField("InvoiceNo", IntegerType(), True), \
                     StructField("StockCode", IntegerType(), True), \
                     StructField("Description", StringType(), True), \
                     StructField("Quantity", IntegerType(), True), \
                     StructField("InvoiceData", StringType(), True), \
                     StructField("Amount", FloatType(), True), \
                     StructField("CustomerID", FloatType(), True), \
                     StructField("Country", StringType(), True)])


In [ ]:
df = spark.read.schema(schema).option("delimiter", ";").csv("retail-data-full.csv")

In [ ]:
df.createOrReplaceTempView("retail_data_full")
results = spark.sql("SELECT CustomerID, min(Amount) FROM retail_data_full GROUP BY CustomerID ORDER BY CustomerID").collect()

In [ ]:
for result in results:
  print(result)

OR

In [ ]:
minAmount = df.select("CustomerID", "Amount")
results = minAmount.groupBy("CustomerID").min("Amount").sort("CustomerID").collect()

In [ ]:
for result in results:
  print(result)

In [ ]:
spark.stop()